In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob
import matplotlib.path as mpath
import matplotlib.colors as col
from netCDF4 import Dataset


import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

figdir = '/g/data/e14/pc5520/figures/basal_melt_param/'

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/36341/status,
Dashboard: /proxy/36341/status,Workers: 5
Total threads: 20,Total memory: 128.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46567,Workers: 5
Dashboard: /proxy/36341/status,Total threads: 20
Started: Just now,Total memory: 128.00 GiB
Comm: tcp://127.0.0.1:44197,Total threads: 4
Dashboard: /proxy/42571/status,Memory: 25.60 GiB
Nanny: tcp://127.0.0.1:46683,


In [3]:
# database for control simulation:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
control = '01deg_jra55v13_ryf9091'

# database for perturbations:
basal_session = cc.database.create_session('/g/data/v45/akm157/model_data/access-om2/01deg_jra55v13_ryf9091_easterlies_up10/easterlies.db')
# 10% increase in easterly wind speed:
easterlies_up = '01deg_jra55v13_ryf9091_easterlies_up10'
# 10% decrease in easterly wind speed:
easterlies_down = '01deg_jra55v13_ryf9091_easterlies_down10'

In [4]:
master_session = cc.database.create_session()

control = '01deg_jra55v13_ryf9091'

# salterature of meltwater at the freezing point calculated with the in situ salinity:
Tfreezing_insituS = 'accessom2-GPC004'
# temperature of meltwater at the freezing point calculated with the 0 salinity:
Tfreezing_zeroS = 'accessom2-GPC005'
# temperature of meltwater equal to the minimum of the above two temperatures:
Tfreezing_min = 'accessom2-GPC006'

start_time='2150-01-01'
end_time='2155-01-01'

lat_slice  = slice(-80,-59)

In [5]:
# build database for these experiments:
session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
basal_melt_session = cc.database.create_session(session_name)

dir_list = ['/g/data/e14/pc5520/access-om2/archive/01deg_jra55v13_ryf9091_rerun_for_easterlies/accessom2-GPC004/',
           '/g/data/e14/pc5520/access-om2/archive/01deg_jra55v13_ryf9091_rerun_for_easterlies/accessom2-GPC005/',
           '/g/data/e14/pc5520/access-om2/archive/01deg_jra55v13_ryf9091_rerun_for_easterlies/accessom2-GPC006/',]
cc.database.build_index(dir_list,basal_melt_session)

Indexing experiment: accessom2-GPC004
Indexing experiment: accessom2-GPC005
Indexing experiment: accessom2-GPC006


100%|██████████| 142/142 [00:12<00:00, 11.45it/s]


142

In [6]:
# topography data for plotting:
ht = cc.querying.getvar(control,'ht' , master_session,n=1).sel(yt_ocean=lat_slice)
land_mask = np.squeeze(ht.values)*0
land_mask[np.isnan(land_mask)] = 1
land_mask = np.where(land_mask==1,land_mask,np.nan)
land_mask_masked = np.ma.masked_where((land_mask==0),land_mask)
# make land go all the way to -90S:
land_mask_lat = ht.yt_ocean.values
land_mask_lat[0] = -90

fontsize=15

In [10]:
#Surface salt and temp

In [7]:
start_time='2150-01-01'
end_time='2155-01-01'

#Import surface_salt
surface_salt_con = cc.querying.getvar(control, 'surface_salt', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time)
surface_salt_con = surface_salt_con.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)

surface_salt_Tfreezing_min = cc.querying.getvar(Tfreezing_min, 'surface_salt', basal_melt_session, frequency='1 monthly',start_time=start_time, end_time=end_time)
surface_salt_Tfreezing_min = surface_salt_Tfreezing_min.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)

In [11]:
start_time='2150-01-01'
end_time='2155-01-01'

#Import surface temp
temp_con = cc.querying.getvar(control, 'temp', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time) - 273.15
temp_con = temp_con.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)

temp_Tfreezing_min = cc.querying.getvar(Tfreezing_min, 'temp', basal_melt_session, frequency='1 monthly',start_time=start_time, end_time=end_time) - 273.15
temp_Tfreezing_min = temp_Tfreezing_min.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)

In [8]:
def plot_surf_salt(tt=0):
    scale_max = .15

    fig  = plt.figure(1, figsize = (25,12))
    norm = col.Normalize(vmin=-scale_max,vmax=scale_max)

    # Tfreezing_min
    ax = fig.add_axes([0.62, 0.0, 0.3, 1],projection=ccrs.SouthPolarStereo())
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    # plot:
    cf = ax.pcolormesh(surface_salt_con_ave.xt_ocean, surface_salt_con_ave.yt_ocean, surface_salt_Tfreezing_min[tt]-surface_salt_con[tt], norm = norm, 
                   cmap = cmocean.balance, transform = ccrs.PlateCarree())
    ax.contour(ht.xt_ocean,ht.yt_ocean,ht,levels=[1000,3000],colors='k',linewidths=0.5,transform = ccrs.PlateCarree())
    plt.title('Tfreezing_min - Control',fontsize=fontsize)
    ax.contourf(ht.xt_ocean, land_mask_lat,land_mask_masked,colors='darkgrey',
             zorder=2,transform = ccrs.PlateCarree())
    ax.set_extent([-180, 180, -90, -59.5], ccrs.PlateCarree())

    # colorbar:
    cbaxes = fig.add_axes([0.93, 0.25, 0.012, 0.5])
    cbar = plt.colorbar(cf, cax = cbaxes, orientation = 'vertical',extend = 'both')
    cbar.set_label(r'Surface salt change',fontsize=fontsize,labelpad=2)
    cbar.ax.tick_params(labelsize=fontsize)

    # save frame
    filestr = figdir+'/surface/salt/image%03d.png' % tt
    plt.savefig(filestr,dpi=150,bbox_inches='tight')
    plt.clf()

In [ ]:
def plot_surf_temp(tt=0):
    scale_max = .4

    fig  = plt.figure(1, figsize = (25,12))
    norm = col.Normalize(vmin=-scale_max,vmax=scale_max)

    # Tfreezing_min
    ax = fig.add_axes([0.62, 0.0, 0.3, 1],projection=ccrs.SouthPolarStereo())
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    # plot:
    cf = ax.pcolormesh(surface_temp_con_ave.xt_ocean, surface_temp_con_ave.yt_ocean, surface_temp_Tfreezing_min[tt]-surface_temp_con[tt], norm = norm, 
                   cmap = cmocean.balance, transform = ccrs.PlateCarree())
    ax.contour(ht.xt_ocean,ht.yt_ocean,ht,levels=[1000,3000],colors='k',linewidths=0.5,transform = ccrs.PlateCarree())
    plt.title('Tfreezing_min - Control',fontsize=fontsize)
    ax.contourf(ht.xt_ocean, land_mask_lat,land_mask_masked,colors='darkgrey',
             zorder=2,transform = ccrs.PlateCarree())
    ax.set_extent([-180, 180, -90, -59.5], ccrs.PlateCarree())

    # colorbar:
    cbaxes = fig.add_axes([0.93, 0.25, 0.012, 0.5])
    cbar = plt.colorbar(cf, cax = cbaxes, orientation = 'vertical',extend = 'both')
    cbar.set_label(r'Surface temp change',fontsize=fontsize,labelpad=2)
    cbar.ax.tick_params(labelsize=fontsize)

    # save frame
    filestr = figdir+'/surface/temp/image%03d.png' % tt
    plt.savefig(filestr,dpi=150,bbox_inches='tight')
    plt.clf()

In [9]:
plot_surf_salt(tt=0)

<Figure size 1800x864 with 0 Axes>

In [18]:
for tt in range(0,59):
    plot_surf(tt=tt)
    print(tt)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


<Figure size 1800x864 with 0 Axes>